# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [2]:
%%writefile ../flows/cloud/taxiprediction_champion.py
from metaflow import FlowSpec, step, card, S3,conda_base, project, retry, catch, current, timeout, Parameter, Flow, trigger
from metaflow.cards import Markdown, Table, Image, Artifact

URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2'})
@project(name="fare_pred")
class TaxiFarePredictionFinal(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):

        obviously_bad_data_filters = [

            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.passenger_count > 0,
            df.tip_amount >= 0,
        ]

        for f in obviously_bad_data_filters:
            df = df[f]    
        return df
    
    @timeout(minutes=5)
    @catch(var="read_failure")
    @retry(times=4)
    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)

        self.df = self.transform_features(df)

        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression

        self.model = LinearRegression()

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Final Week 4 Results - Champion"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePredictionFinal()

Overwriting ../flows/cloud/taxiprediction_champion.py


In [3]:
! python ../flows/cloud/taxiprediction_champion.py --environment=conda run 

Metaflow 2.9.1+ob(v1) executing TaxiFarePredictionFinal for user:sandbox
Project: fare_pred, Branch: user.sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Bootstrapping conda environment...(this could take a few minutes)
2023-05-21 10:11:09.923 Workflow starting (run-id 98), see it in the UI at https://ui-pw-456654186.outerbounds.dev/TaxiFarePredictionFinal/98
2023-05-21 10:11:10.816 [98/start/376 (pid 22981)] Task is starting.
2023-05-21 10:11:17.672 [98/start/376 (pid 22981)] Task finished successfully.
2023-05-21 10:11:18.640 [98/linear_model/377 (pid 23066)] Task is starting.
2023-05-21 10:11:21.147 [98/linear_model/377 (pid 23066)] Task finished successfully.
2023-05-21 10:11:22.007 [98/validate/378 (pid 23110)] Task is starting.
2023-05-21 10:11:27.414 [98/validate/378 (pid 23110)] Task finished successfully.
2023-05-21 10:11:28.362 [98/end/379 (pid 23170)] Task is starting.
2023-05-21 10:11:29.292 [98/end/379 (pid 23170)] Success!


In [4]:
! python ../flows/cloud/taxiprediction_champion.py --environment=conda --production --branch champion argo-workflows create 

Metaflow 2.9.1+ob(v1) executing TaxiFarePredictionFinal for user:sandbox
Project: fare_pred, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying farepred.prod.champion.taxifarepredictionfinal to Argo Workflows...

The namespace of this production flow is
    production:mfprj-egowotrszwf6iixq-0-tieg
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-egowotrszwf6iixq-0-tieg")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-egowotrszwf6iixq-0-tieg
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow farepred.prod.champion.taxifarepredictionfinal for flow TaxiFarePredictionFina

## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [5]:
%%writefile ../flows/cloud/taxiprediction_challenger.py
from metaflow import FlowSpec, step, card, S3,conda_base, project, current, retry, catch, timeout, Parameter, Flow, trigger
from metaflow.cards import Markdown, Table, Image, Artifact

URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2', 'xgboost' : '1.7.4'})
@project(name="fare_pred")
class TaxiFarePredictionFinal(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):

        obviously_bad_data_filters = [

            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.passenger_count > 0,
            df.tip_amount >= 0,
        ]

        for f in obviously_bad_data_filters:
            df = df[f]    
        return df
    
    @timeout(minutes=5)
    @catch(var="read_failure")
    @retry(times=2)
    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)

        self.df = self.transform_features(df)

        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.xgb_reg)

    @step
    def xgb_reg(self):
        "Fit a single variable, linear model to the data."
        from xgboost import XGBRegressor

        self.model = XGBRegressor(n_estimators=100)

        self.next(self.validate)

    def gather_sibling_flow_run_results(self):

        # storage to populate and feed to a Table in a Metaflow card
        rows = []

        # loop through runs of this flow 
        for run in Flow(self.__class__.__name__):
            if run.id != current.run_id:
                if run.successful:
                    icon = "✅" 
                    msg = "OK"
                    score = str(run.data.scores.mean())
                else:
                    icon = "❌"
                    msg = "Error"
                    score = "NA"
                    for step in run:
                        for task in step:
                            if not task.successful:
                                msg = task.stderr
                row = [Markdown(icon), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(score), Markdown(msg)]
                rows.append(row)
            else:
                rows.append([Markdown("✅"), Artifact(run.id), Artifact(run.created_at.strftime(DATETIME_FORMAT)), Artifact(str(self.scores.mean())), Markdown("This run...")])
        return rows
                
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Final Week 4 Results - challenger"))
        current.card.append(Table(self.gather_sibling_flow_run_results(), headers=["Pass/fail", "Run ID", "Created At", "R^2 score", "Stderr"]))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePredictionFinal()

Overwriting ../flows/cloud/taxiprediction_challenger.py


In [6]:
! python ../flows/cloud/taxiprediction_challenger.py --environment=conda run 

Metaflow 2.9.1+ob(v1) executing TaxiFarePredictionFinal for user:sandbox
Project: fare_pred, Branch: user.sandbox
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Bootstrapping conda environment...(this could take a few minutes)
2023-05-21 10:11:46.802 Workflow starting (run-id 99), see it in the UI at https://ui-pw-456654186.outerbounds.dev/TaxiFarePredictionFinal/99
2023-05-21 10:11:47.796 [99/start/381 (pid 23293)] Task is starting.
2023-05-21 10:11:53.707 [99/start/381 (pid 23293)] Task finished successfully.
2023-05-21 10:11:54.645 [99/xgb_reg/382 (pid 23378)] Task is starting.
2023-05-21 10:11:57.569 [99/xgb_reg/382 (pid 23378)] Task finished successfully.
2023-05-21 10:11:58.517 [99/validate/384 (pid 23424)] Task is starting.
2023-05-21 10:12:58.445 [99/validate/384 (pid 23424)] Task finished successfully.
2023-05-21 10:12:59.416 [99/end/388 (pid 23491)] Task is starting.
2023-05-21 10:13:00.423 [99/end/388 (pid 23491)] Success!
2023-05-21

In [7]:
! python ../flows/cloud/taxiprediction_challenger.py --environment=conda --production --branch challenger argo-workflows create 

Metaflow 2.9.1+ob(v1) executing TaxiFarePredictionFinal for user:sandbox
Project: fare_pred, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying farepred.prod.challenger.taxifarepredictionfinal to Argo Workflows...

The namespace of this production flow is
    production:mfprj-cxk5tyqshp7w6yi2-0-qxvi
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-cxk5tyqshp7w6yi2-0-qxvi")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-cxk5tyqshp7w6yi2-0-qxvi
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow farepred.prod.challenger.taxifarepredictionfinal for flow TaxiFarePredicti

In [8]:
! python ../flows/cloud/taxiprediction_champion.py --environment=conda --production --branch champion --production argo-workflows trigger
! python ../flows/cloud/taxiprediction_challenger.py --environment=conda --production --branch challenger --production argo-workflows trigger

Metaflow 2.9.1+ob(v1) executing TaxiFarePredictionFinal for user:sandbox
Project: fare_pred, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow farepred.prod.champion.taxifarepredictionfinal triggered on Argo Workflows (run-id argo-farepred.prod.champion.taxifarepredictionfinal-kbtgd).
See the run in the UI at https://ui-pw-456654186.outerbounds.dev/TaxiFarePredictionFinal/argo-farepred.prod.champion.taxifarepredictionfinal-kbtgd
Metaflow 2.9.1+ob(v1) executing TaxiFarePredictionFinal for user:sandbox
Project: fare_pred, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow farepred.prod.challenger.taxifarepredictionfinal triggered on Argo Workflows (run-id argo-farepred.prod.challenger.taxifarepredictionfinal-985dk).
See the run in the UI at https://ui-pw-456654186.outerbounds.dev/TaxiFarePredictionFinal/argo-farepred.prod.challenger.taxifarepredicti

## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected?
Yes!

- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B? 

    Based on the chosen metric, which is the R^2 score, the challenger model appears to perform marginally better. However, there are other issues to be analyzed, such as inference latency, consumed resource and consistency, which have not been analyzed so far.

In [9]:
from metaflow import Flow, namespace
import numpy as np

CHAMPION_NAMESPACE = 'production:mfprj-egowotrszwf6iixq-0-tieg'
CHALLENGER_NAMESPACE = 'production:mfprj-cxk5tyqshp7w6yi2-0-qxvi'

for space, name in zip([CHAMPION_NAMESPACE, CHALLENGER_NAMESPACE],['champion','challenger']):
    namespace(space)
    run = Flow('TaxiFarePredictionFinal').latest_successful_run
    acc_score = np.mean(run.data.scores)
    acc_score_sd = np.std(run.data.scores)
    print("{} model had accuracy = {}% ({})".format(name, acc_score,acc_score_sd))


champion model had accuracy = 0.896859637689513% (0.003105554078294598)
challenger model had accuracy = 0.9052422318985247% (0.002526478542924083)


## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!